## 留言板 (進度)

12/08 (彥)
- TTA 在b1沒用，其他應該也是
- 不知道為啥凍結比沒凍結效能好，可能是因為資料集太少，重新學習沒效率跟沒啥用
- 骨幹學習率還可以調看看 1e-6，可能這樣凍結就有效了
- weight_decay、label_smoothing、AutoAugmented 看起來很沒用

12/07 (彥)
- 我把大部分程式碼寫上備註了，然後區塊應該也分好了。要修改哪些操作變數的話就直接按旁邊的目錄可以直接跳過去！
- 然後程式碼的話可以自己跑看看，因為我執行結果印象有點模糊@@ (efficient_b1 我只有跑全部因子都使用的) 
- 阿做了啥可以直接在這邊寫方便回憶一下，加油加油！ 
- 阿會不會覺得模型有點太多了
 <br>


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2025-iyatomi-lab-competition/sample_submission.csv
/kaggle/input/2025-iyatomi-lab-competition/test/208.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/029.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/473.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/014.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/333.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/537.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/369.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/275.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/212.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/239.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/150.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/109.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/034.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/149.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/187.jpg
/kaggle/input/2025-iyatomi-lab-competition/test/521.jpg
/kaggle/input/2025-iyatomi-lab-competit

In [2]:
# 下載熱力圖套件
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.2 MB/s eta 0:00:00:00:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00:

In [3]:
import time
import math
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, Dataset
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2

## 參數調整

In [4]:
# 資料集匯入
BASE_DIR = '/kaggle/input/2025-iyatomi-lab-competition' 
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
TEST_DIR = os.path.join(BASE_DIR, 'test')
SAMPLE_CSV_PATH = os.path.join(BASE_DIR, 'sample_submission.csv')

# 參數區
SEED_VALUE = 42
VALID_SIZE = 0.1
INPUT_SIZE = 224  

BATCH_SIZE = 16

LEARNING_RATE = 1e-3
WEIGHT_DECAY_RATE = 1e-4
EPOCHS = 60

torch.manual_seed(SEED_VALUE)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED_VALUE)

In [ ]:
# 1. CBAM 模組定義 (針對 CNN 使用)
# ==========================================
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.ca(x) * x
        x = self.sa(x) * x
        return x

# ==========================================
# 2. CNN 包裝器 (把 CNN 骨幹 + CBAM 組合起來)
# ==========================================
class CNNWithCBAM(nn.Module):
    def __init__(self, features, in_features, num_classes):
        super(CNNWithCBAM, self).__init__()
        self.features = features
        # 加入 CBAM
        self.cbam = CBAM(in_planes=in_features)
        # 新的分類頭
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=0.3),
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.cbam(x) # 關鍵創新點
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

In [5]:
def get_base_model(model_name, num_classes=120, freeze_backbone=False):
    """
    全能模型工廠：支援 EfficientNet 系列, ViT, Swin Transformer
    
    參數:
    model_name (str): 支援 'efficientnet_b0' ~ 'b4', 'vit_b_16', 'swin_t', 'resnet50'
    """
    print(f"正在建立模型: {model_name} (Freeze: {freeze_backbone})...")
    
    model = None
    input_size = 224 # 預設輸入大小
    
    # ==========================================
    # 1. 下載預訓練模型與權重
    # ==========================================
    
    # --- EfficientNet 系列 ---
    if model_name == "efficientnet_b0":
        weights = models.EfficientNet_B0_Weights.DEFAULT
        model = models.efficientnet_b0(weights=weights)
        input_size = 224
    elif model_name == "efficientnet_b1":
        weights = models.EfficientNet_B1_Weights.DEFAULT
        model = models.efficientnet_b1(weights=weights)
        input_size = 240 # B1 建議 240
    elif model_name == "efficientnet_b3":
        weights = models.EfficientNet_B3_Weights.DEFAULT
        model = models.efficientnet_b3(weights=weights)
        input_size = 300 # B3 建議 300
    elif model_name == "efficientnet_b4":
        weights = models.EfficientNet_B4_Weights.DEFAULT
        model = models.efficientnet_b4(weights=weights)
        input_size = 380 # B4 建議 380 (這很吃顯存！)
        
    # --- Vision Transformer (ViT) 系列 ---
    elif model_name == "vit_b_16":
        # 最經典的 ViT Base model, patch size 16
        weights = models.ViT_B_16_Weights.DEFAULT
        model = models.vit_b_16(weights=weights)
        input_size = 224
        
    # --- Swin Transformer 系列 ---
    elif model_name == "swin_t":
        # Swin Transformer Tiny (效能接近 ResNet50 但更強)
        weights = models.Swin_T_Weights.DEFAULT
        model = models.swin_t(weights=weights)
        input_size = 224
        
    # --- ResNet 系列 ---
    elif model_name == "resnet50":
        weights = models.ResNet50_Weights.DEFAULT
        model = models.resnet50(weights=weights)
        input_size = 224
        
    else:
        raise ValueError(f"不支援的模型名稱: {model_name}")

    # ==========================================
    # 2. 凍結骨幹 (如果需要)
    # ==========================================
    if freeze_backbone:
        print("   -> 凍結所有骨幹參數...")
        for param in model.parameters():
            param.requires_grad = False
    else:
        # --- 針對不同模型做「部分解凍」 ---
        
        # A. 如果是 ResNet (這裡是你原本報錯的地方)
        if 'resnet' in model_name:
            print("   -> 偵測到 ResNet，先凍結全部，再解凍最後兩層 (Layer3, Layer4)...")
            
            # 1. 先把全部鎖起來
            for param in model.parameters():
                param.requires_grad = False
            
            # 2. 解凍 Layer4 (最後一層卷積)
            for param in model.layer4.parameters():
                param.requires_grad = True
                
            # 3. 解凍 Layer3 (倒數第二層卷積)
            #for param in model.layer3.parameters():
            #    param.requires_grad = True
                
        # B. 如果是 EfficientNet (保留你原本的寫法)
        elif 'efficientnet' in model_name:
            print("   -> 偵測到 EfficientNet，正在解凍最後幾個 Block...")
            
            # 1. 先全部鎖起來
            for param in model.features.parameters():
                param.requires_grad = False
            
            # 2. 算一下要解凍哪裡
            num_blocks_to_unfreeze = 2
            total_blocks = len(model.features)
            start_index = total_blocks - num_blocks_to_unfreeze
            
            for i in range(start_index, total_blocks):
                print(f"      -> 解凍 features[{i}]...")
                for param in model.features[i].parameters():
                    param.requires_grad = True
        
        # C. 其他模型 (ViT, Swin)
        else:
            print("   -> 其他模型 (ViT/Swin)，預設全部解凍以避免錯誤。")
            for param in model.parameters():
                param.requires_grad = True    
    
    # ==========================================
    # 3. 修改分類頭 (針對不同架構自動調整)
    # ==========================================
    
    # A. 針對 EfficientNet 系列
    if "efficientnet" in model_name:
        in_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(p=0.3, inplace=True), # 稍微調高 dropout
            nn.Linear(in_features, num_classes)
        )
        
    # B. 針對 ResNet 系列
    elif "resnet" in model_name:
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)
        
    # C. 針對 ViT (Vision Transformer)
    elif "vit" in model_name:
        # ViT 的頭叫做 'heads'
        in_features = model.heads.head.in_features 
        # 重建 heads (ViT 通常不需要太多 Dropout，直接接 Linear 即可)
        model.heads = nn.Sequential(
            nn.Linear(in_features, num_classes)
        )
        
    # D. 針對 Swin Transformer
    elif "swin" in model_name:
        # Swin 的頭叫做 'head'
        in_features = model.head.in_features
        model.head = nn.Linear(in_features, num_classes)

    print(f"已成功建立模型 : {model_name}")

    return model, input_size

# --- 使用範例 ---
# 這樣呼叫時，它還會告訴你這個模型適合的 input_size 是多少
# model, img_size = get_base_model("efficientnet_b4", num_classes=120)
# print(f"模型建立完成，請將圖片 Resize 成: {img_size}")

In [ ]:
def get_advanced_model(model_name, num_classes=120, freeze_backbone=False):
    """
    全能工廠：
    - CNN 系列 (EfficientNet/ResNet) -> 自動加裝 CBAM
    - Transformer 系列 (ViT/Swin) -> 使用原生 Self-Attention
    """
    print(f"正在建立模型: {model_name} ...")
    
    model = None
    input_size = 224 # 預設
    
    # ---------------------------
    # A. EfficientNet 系列 (加上 CBAM)
    # ---------------------------
    if "efficientnet" in model_name:
        if model_name == "efficientnet_b0":
            weights = models.EfficientNet_B0_Weights.DEFAULT
            base = models.efficientnet_b0(weights=weights)
            input_size = 224
        elif model_name == "efficientnet_b1":
            weights = models.EfficientNet_B1_Weights.DEFAULT
            base = models.efficientnet_b1(weights=weights)
            input_size = 240
        elif model_name == "efficientnet_b3":
            weights = models.EfficientNet_B3_Weights.DEFAULT
            base = models.efficientnet_b3(weights=weights)
            input_size = 300
        elif model_name == "efficientnet_b4":
            weights = models.EfficientNet_B4_Weights.DEFAULT
            base = models.efficientnet_b4(weights=weights)
            input_size = 380
        else:
            raise ValueError("目前只支援 b0, b1, b3, b4")
            
        # 提取特徵層與通道數
        features = base.features
        in_features = base.classifier[1].in_features
        
        # 包裝成帶有 CBAM 的模型
        model = CNNWithCBAM(features, in_features, num_classes)
        print(f"-> 已啟用架構創新：{model_name} + CBAM Module")

    # ---------------------------
    # B. ResNet 系列 (加上 CBAM)
    # ---------------------------
    elif "resnet" in model_name:
        if model_name == "resnet50":
            weights = models.ResNet50_Weights.DEFAULT
            base = models.resnet50(weights=weights)
            input_size = 224
            
        # ResNet 比較特別，要去掉最後兩層 (AvgPool 和 FC) 才能拿到特徵圖
        features = nn.Sequential(*list(base.children())[:-2])
        in_features = base.fc.in_features
        
        # 包裝
        model = CNNWithCBAM(features, in_features, num_classes)
        print(f"-> 已啟用架構創新：{model_name} + CBAM Module")

    # ---------------------------
    # C. Transformer 系列 (ViT, Swin)
    # ---------------------------
    elif "vit" in model_name or "swin" in model_name:
        if model_name == "vit_b_16":
            weights = models.ViT_B_16_Weights.DEFAULT
            model = models.vit_b_16(weights=weights)
            in_features = model.heads.head.in_features
            model.heads = nn.Sequential(nn.Linear(in_features, num_classes))
            input_size = 224
            
        elif model_name == "swin_t":
            weights = models.Swin_T_Weights.DEFAULT
            model = models.swin_t(weights=weights)
            in_features = model.head.in_features
            model.head = nn.Linear(in_features, num_classes)
            input_size = 224
            
        print(f"-> Transformer 架構檢測：使用原生 Self-Attention (不需 CBAM)")

    else:
        raise ValueError(f"不支援的模型: {model_name}")

    # ---------------------------
    # D. 處理凍結
    # ---------------------------
    if freeze_backbone:
        print("凍結骨幹！")
        # 如果是我們包裝過的 CNNWithCBAM，骨幹在 model.features
        if hasattr(model, 'features'):
            for param in model.features.parameters():
                param.requires_grad = False
        # 如果是 Transformer，直接鎖定所有參數，除了最後一層
        else:
            for name, param in model.named_parameters():
                if "head" not in name: # 只解鎖含有 head (分類頭) 的層
                    param.requires_grad = False
    else:
        print("-> 啟用漸進式微調策略：凍結骨幹前面部分！")
    
    # 如果是我們包裝過的 CNNWithCBAM (EfficientNet/ResNet)
        if hasattr(model, 'features'):
            num_blocks_to_unfreeze = 2 # 解凍最後 4 層
            for param in model.features.parameters():
                param.requires_grad = False
            total_blocks = len(model.features) # 獲取骨幹總 Block 數
            start_index = total_blocks - num_blocks_to_unfreeze

            # 遍歷從 start_index 到最後一個 Block
            for i in range(start_index, total_blocks):
                print(f"   -> 解凍 model.features[{i}] Block...")
                # 遍歷被選中的 Block，將參數解凍 (設為 True)
                for param in model.features[i].parameters():
                    param.requires_grad = True
                
            # 注意：model.cbam 和 model.classifier 預設是 requires_grad = True，會保持解凍狀態。
        
        # 如果是 Transformer，使用原本的邏輯（只鎖定 head 以外的部分）
        else:
            print("-> Transformer 模型，只解鎖分類頭。")
            for name, param in model.named_parameters():
                if "head" not in name: 
                    param.requires_grad = False

    return model, input_size

## 模型選擇
基礎模型就是沒加上 CBAM 或其他輔助 <br>
強化模型就是有 <br>
 <br>
 選一個跑就好！

In [6]:
# 基礎版(預訓練模型)
# 建立模型(選擇模型)
# freeze_backbone : 是否凍結原始參數
#model, img_size = get_base_model("efficientnet_b0", num_classes=120, freeze_backbone=False)
#model, img_size = get_base_model("efficientnet_b1", num_classes=120, freeze_backbone=False)
#model, img_size = get_base_model("efficientnet_b3", num_classes=120, freeze_backbone=False)
#model, img_size = get_base_model("efficientnet_b4", num_classes=120, freeze_backbone=False)
#model, img_size = get_base_model("resnet50", num_classes=120, freeze_backbone=True)
model, img_size = get_base_model("vit_b_16", num_classes=120, freeze_backbone=True)
#model, img_size = get_base_model("swin_t", num_classes=120, freeze_backbone=True)

正在建立模型: vit_b_16 (Freeze: True)...


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 212MB/s] 


   -> 凍結所有骨幹參數...
已成功建立模型 : vit_b_16


In [ ]:
# 強化版( + CBAM or Attention)
# 建立模型(選擇模型)
# freeze_backbone : 是否凍結原始參數
#model, img_size = get_advanced_model("efficientnet_b0", num_classes=120, freeze_backbone=False)
#model, img_size = get_advanced_model("efficientnet_b1", num_classes=120, freeze_backbone=False)
#model, img_size = get_advanced_model("efficientnet_b3", num_classes=120, freeze_backbone=False)
#model, img_size = get_advanced_model("efficientnet_b4", num_classes=120, freeze_backbone=False)
model, img_size = get_advanced_model("resnet50", num_classes=120, freeze_backbone=False)
#model, img_size = get_advanced_model("vit_b_16", num_classes=120, freeze_backbone=False)
#model, img_size = get_advanced_model("swim_t", num_classes=120, freeze_backbone=True)

## Loss Function
crition 看 label_smoothing 要不要關掉 <br>
optimizer 看 weight_decay 要不要關掉 <br>
schdeular 看要不要關掉 <br>
 <br>
這些都是操作變因！

In [7]:
# Base模型用這個
# 把模型搬到 GPU (如果有的話)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 定義損失函數
criterion = nn.CrossEntropyLoss(
    label_smoothing=0.1
)

# 定義優化器
# 【關鍵】使用 filter 只抓出需要訓練的參數 (也就是最後那一層)
optimizer = optim.RAdam(
    filter(lambda p: p.requires_grad, model.parameters()), 
    lr=LEARNING_RATE,
    #weight_decay=1e-4
)
# T_max=EPOCHS: 告訴它我們總共要跑多少回合，讓它規劃變速節奏
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

print("模型準備完成！現在是原始版本。")
print(f"使用裝置: {device}")

模型準備完成！現在是原始版本。
使用裝置: cuda


In [ ]:
# Advanced模型用這個
# 把模型搬到 GPU (如果有的話)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 定義損失函數
criterion = nn.CrossEntropyLoss(
    label_smoothing=0.1
)

# 定義兩種學習率
# 沿用你目前成功的 LR 作為新層的 LR
lr_head = LEARNING_RATE       # 假設為 1e-3 (用於 CBAM 和 Classifier)
# 將骨幹的 LR 設得極低，例如低 100 倍，進行溫和微調
lr_base = LEARNING_RATE * 0.01 # 例如 1e-5 (用於微調 features)

# 步驟 2: 建立參數組列表
param_groups = [
    # 組別 1：高學習率 (CBAM 和 分類頭) - 快速學習
    {
        'params': list(model.cbam.parameters()) + list(model.classifier.parameters()), 
        'lr': lr_head,
        'weight_decay': WEIGHT_DECAY_RATE
    },
    
    # 組別 2：低學習率 (微調骨幹 Features) - 溫和微調
    {
        # 【關鍵】這裡仍使用 filter，因為只有 model.features 的最後幾層 requires_grad=True
        'params': filter(lambda p: p.requires_grad, model.features.parameters()), 
        'lr': lr_base,
        'weight_decay': WEIGHT_DECAY_RATE
    }
]

# 定義優化器
# 【關鍵】使用 filter 只抓出需要訓練的參數 (也就是最後那一層)
optimizer = optim.RAdam(
    param_groups,
)

# T_max=EPOCHS: 告訴它我們總共要跑多少回合，讓它規劃變速節奏
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

print("模型準備完成！現在是 EfficientNet 版本。")
print(f"使用裝置: {device}")

## 資料預處理
一般 : 裁切、翻轉 <br>
進階 : AutoAugmented <br>
 <br>
選一個做就好！

In [8]:
# 定義資料處理 SOP
# 裁切、翻轉
data_transforms = transforms.Compose([
    # 這裡會自動用到上面的 INPUT_SIZE (img_size)
    transforms.RandomResizedCrop(size=(img_size, img_size), scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# 定義資料處理 SOP
# AutoAugmented
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(size=(img_size, img_size), scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(),
    
    # 【新增這行】AutoAugment (ImageNet 策略)
    # 這招非常強，會大幅增加訓練難度
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
print("正在準備資料...")

full_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=data_transforms)

# 切分訓練集與驗證集
val_size = int(len(full_dataset) * VALID_SIZE)
train_size = len(full_dataset) - val_size
generator = torch.Generator().manual_seed(SEED_VALUE)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

# 建立 DataLoader (搬運工)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"資料準備完成！訓練集: {len(train_dataset)}, 驗證集: {len(val_dataset)}")

正在準備資料...
資料準備完成！訓練集: 1202, 驗證集: 133


## Test Datasets

In [10]:
# 1. 定義一個專門讀取測試圖片的工具 (Dataset)
class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = test_dir
        self.transform = transform
        # 抓出資料夾裡所有的 jpg/png 圖片
        self.image_files = [f for f in os.listdir(test_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # 拿第 idx 張圖
        img_name = self.image_files[idx]
        img_path = os.path.join(self.test_dir, img_name)
        
        # 讀取圖片並轉成 RGB (預防有黑白圖混進來報錯)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        # 回傳：處理好的圖片 (給模型看), 檔名 (讓你之後填 CSV 用)
        return image, img_name

test_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    # 記得也要標準化，不然模型會看不懂
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. 建立 test_loader
# 這裡 batch_size 可以設大一點沒關係，因為考試不用算梯度，比較省記憶體
test_dataset = TestDataset(TEST_DIR, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"成功建立 test_loader！")
print(f"測試集位置: {TEST_DIR}")
print(f"總共有 {len(test_dataset)} 張圖片等待考試。")

成功建立 test_loader！
測試集位置: /kaggle/input/2025-iyatomi-lab-competition/test
總共有 573 張圖片等待考試。


## 模型訓練

In [11]:
num_epochs = EPOCHS 

print(f"開始訓練！總共要跑 {num_epochs} 個 Epoch (回合)...")
print("-" * 50)

# 儲存最佳成績，用來存擋
best_acc = 0.0

best_loss = float('inf')  # 追蹤歷史最低驗證損失
patience = 10              # 允許連續 10 輪驗證損失不下降
patience_counter = 0      # 算計數器

for epoch in range(num_epochs):
    start_time = time.time() # 計時開始
    
    # ==========================
    # 1. 訓練模式 (讀書時間)
    # ==========================
    model.train() # 切換開關：告訴模型現在是「訓練模式」 (要啟用 Dropout 等機制)
    
    running_loss = 0.0
    running_corrects = 0
    
    # 從 train_loader 一批一批拿資料出來
    for inputs, labels in train_loader:
        # 把資料搬到 GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # 【標準動作 1】歸零：把上一題的計算痕跡清掉
        optimizer.zero_grad()
        
        # 【標準動作 2】考試：模型看圖猜答案 (Forward)
        outputs = model(inputs)
        
        # 【標準動作 3】對答案：計算猜錯多少 (Loss)
        loss = criterion(outputs, labels)
        
        # 【標準動作 4】找原因：回頭看是哪裡算錯了 (Backward)
        loss.backward()
        
        # 【標準動作 5】修正：根據錯誤修改參數 (Update)
        optimizer.step()
        
        # --- 統計成績 (只是為了印出來看，不影響訓練) ---
        running_loss += loss.item() * inputs.size(0)
        # 找出分數最高的那個類別當作預測結果
        _, preds = torch.max(outputs, 1) 
        running_corrects += torch.sum(preds == labels.data)

    # 【關鍵】每個 Epoch 結束後，通知變速箱調整學習率
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr'] # CBAM lr
    #current_lr = LEARNING_RATE
    # 計算這一個 Epoch 的平均 Loss 和 準確率
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)

    # ==========================
    # 2. 驗證模式 (模擬考時間)
    # ==========================
    model.eval() # 切換開關：告訴模型現在是「考試模式」 (關閉 Dropout，成績才準)
    
    val_loss = 0.0
    val_corrects = 0
    
    # 考試的時候不需要算梯度 (Gradient)，這樣比較省記憶體也比較快
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)
            
    # 計算驗證集的成績
    val_loss = val_loss / len(val_dataset)
    val_acc = val_corrects.double() / len(val_dataset)

    # Early stop
    if val_loss < best_loss:
        best_loss = val_loss
        patience_counter = 0 # 損失下降了，計數器歸零
        # torch.save(model.state_dict(), 'best_model.pth') # 儲存當前模型
        
        print(f"--> 損失下降，重設耐心值。")
    else:
        patience_counter += 1 # 損失沒有下降，耐心值 +1
        print(f"--> 損失未下降，耐心值: {patience_counter}/{patience}")
    
    # ==========================
    # 3. 印出成績單
    # ==========================
    time_elapsed = time.time() - start_time
    print(f'Epoch {epoch+1}/{num_epochs} | 耗時: {time_elapsed:.0f}秒 | LR: {current_lr:.6f}')
    print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print(f'Val   Loss: {val_loss:.4f}   Acc: {val_acc:.4f}')
    
    # 如果這次考得比歷史最高分還高，就把它存起來 (或是你可以加上存檔的程式碼)
    if val_acc > best_acc:
        best_acc = val_acc
        # torch.save(model.state_dict(), 'best_model.pth') # 想存檔可以把這行打開
        print(f"--> 恭喜！發現新高分 (Best Accuracy): {best_acc:.4f}")

    # 檢查是否達到停止條件
    if patience_counter >= patience:
        print("-" * 50)
        print(f"🚨🚨🚨 提前停止！驗證損失連續 {patience} 輪沒有改善。")
        break # 跳出 for epoch in range(num_epochs): 迴圈
    
    print("-" * 50)

print(f"訓練結束！歷史最高驗證準確率: {best_acc:.4f}")

開始訓練！總共要跑 60 個 Epoch (回合)...
--------------------------------------------------
--> 損失下降，重設耐心值。
Epoch 1/60 | 耗時: 29秒 | LR: 0.000999
Train Loss: 3.9396 Acc: 0.4268
Val   Loss: 2.7956   Acc: 0.7068
--> 恭喜！發現新高分 (Best Accuracy): 0.7068
--------------------------------------------------
--> 損失下降，重設耐心值。
Epoch 2/60 | 耗時: 29秒 | LR: 0.000997
Train Loss: 1.8903 Acc: 0.7945
Val   Loss: 1.3290   Acc: 0.8647
--> 恭喜！發現新高分 (Best Accuracy): 0.8647
--------------------------------------------------
--> 損失下降，重設耐心值。
Epoch 3/60 | 耗時: 31秒 | LR: 0.000994
Train Loss: 1.2179 Acc: 0.8627
Val   Loss: 1.1704   Acc: 0.8647
--------------------------------------------------
--> 損失下降，重設耐心值。
Epoch 4/60 | 耗時: 32秒 | LR: 0.000989
Train Loss: 1.1204 Acc: 0.8727
Val   Loss: 1.1292   Acc: 0.8872
--> 恭喜！發現新高分 (Best Accuracy): 0.8872
--------------------------------------------------
--> 損失下降，重設耐心值。
Epoch 5/60 | 耗時: 35秒 | LR: 0.000983
Train Loss: 1.0868 Acc: 0.8927
Val   Loss: 1.0747   Acc: 0.8947
--> 恭喜！發現新高分 (Best Accura

## 模型預測 (交上去 kaggle 的)

In [ ]:
print("=" * 50)
print(f"開始對測試集 (Test Set) 進行預測...")
print("目標：產出 submission.csv 以便上傳 Kaggle 計分。")
print("=" * 50)

# 1. 切換到評估模式 (非常重要！關閉 Dropout)
model.eval()

predictions = []
image_ids = []

# 2. 開始考試 (不計算梯度，節省記憶體)
with torch.no_grad():
    # 注意：這裡的 test_loader 回傳的是 (images, filenames)
    # 而不是 (images, labels)，所以不能算 Loss 或 Accuracy
    for i, (inputs, filenames) in enumerate(test_loader):
        inputs = inputs.to(device)
        
        # 模型預測
        outputs = model(inputs)
        
        # 取出分數最高的那個類別 index
        _, preds = torch.max(outputs, 1)
        
        # 存下來
        predictions.extend(preds.cpu().numpy())
        image_ids.extend(filenames)
        
        # (選用) 顯示進度
        if (i + 1) % 10 == 0:
            print(f"已處理 Batch {i+1}/{len(test_loader)}")

# 3. 轉換結果並存檔
print("-" * 50)
print("預測完成！正在轉換格式...")

# 取得類別對照表 (Index -> 狗名)
class_names = full_dataset.classes 

# 把數字轉回狗的名字
predicted_labels = [class_names[p] for p in predictions]

# 處理檔名 (去掉 .jpg 副檔名)
clean_ids = image_ids

# 建立 DataFrame
df_sub = pd.DataFrame({
    'id': clean_ids,
    'label': predicted_labels
})

# 存成 CSV
submission_file = 'submission.csv'
df_sub.to_csv(submission_file, index=False)

print(f"✅ 檔案已建立: {submission_file}")
print("下一步：請下載此檔案並上傳到 Kaggle，就能看到準確度了！")
print("-" * 50)
print("前 5 筆預測預覽：")
print(df_sub.head())

## TTA 模型預測

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image

print("=" * 50)
print("🧐 正在檢查 TTA 對驗證集 (Validation Set) 的效果...")
print("=" * 50)

# ==========================================
# 1. 定義 TTA 變換 (沿用你原本的設定)
# ==========================================
inference_size = img_size # B1 的尺寸 (請確認跟你的模型一致)

def get_tta_transforms(size):
    return [
        # 1. 原圖 (標準)
        transforms.Compose([
            transforms.Resize((size, size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # 2. 水平翻轉
        transforms.Compose([
            transforms.Resize((size, size)),
            transforms.RandomHorizontalFlip(p=1.0), # 強制翻轉
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # 3. 稍微放大 (Center Crop)
        transforms.Compose([
            transforms.Resize(int(size * 1.1)), # 先放大 10%
            transforms.CenterCrop(size),        # 再切中間
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # 4. 稍微向左轉 (Rotation)
        transforms.Compose([
            transforms.Resize((size, size)),
            transforms.RandomRotation(degrees=(10, 10)), # 固定轉 10 度
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # 5. 稍微向右轉
        transforms.Compose([
            transforms.Resize((size, size)),
            transforms.RandomRotation(degrees=(-10, -10)), # 固定轉 -10 度
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
    ]

tta_transforms = get_tta_transforms(inference_size)

# ======================= ===================
# 2. 定義驗證用的 TTA Dataset
# ==========================================
# 這個 Dataset 的功能是：
# 從原本的 val_dataset 找出圖片路徑 -> 打開圖片 -> 做 5 種變換 -> 回傳 (5張圖, 正確答案)
class TTAValidationDataset(Dataset):
    def __init__(self, val_dataset, transforms_list):
        self.transforms_list = transforms_list
        
        # 判斷 val_dataset 是完整的 ImageFolder 還是被切分過的 Subset
        if isinstance(val_dataset, Subset):
            # 如果是 Subset (random_split 切出來的)，我們要透過 indices 找回原本的路徑
            self.samples = [val_dataset.dataset.samples[i] for i in val_dataset.indices]
        elif hasattr(val_dataset, 'samples'):
            # 如果是直接的 ImageFolder
            self.samples = val_dataset.samples
        else:
            # 如果是其他狀況，我們試著直接用 dataset
            self.samples = val_dataset.imgs 

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        path, label = self.samples[idx] # 取得圖片路徑和正確答案
        image = Image.open(path).convert('RGB')
        
        # 產生 5 張變換後的圖
        augmented_images = [t(image) for t in self.transforms_list]
        
        # 堆疊成 (5, 3, 240, 240)
        return torch.stack(augmented_images), label

# ==========================================
# 3. 準備 DataLoader
# ==========================================
# 使用你原本程式碼中的 val_dataset
tta_val_dataset = TTAValidationDataset(val_dataset, tta_transforms)
# batch_size 設小一點 (例如 4 或 8)，因為現在一筆資料等於 5 張圖
tta_val_loader = DataLoader(tta_val_dataset, batch_size=4, shuffle=False, num_workers=2)

# ==========================================
# 4. 開始 TTA 驗證
# ==========================================
model.eval()
tta_corrects = 0
total_samples = 0

print(f"開始評估... (每張圖預測 {len(tta_transforms)} 次取平均)")

with torch.no_grad():
    for i, (inputs, labels) in enumerate(tta_val_loader):
        # inputs: (Batch, 5, 3, H, W)
        # labels: (Batch)
        
        labels = labels.to(device)
        bs, n_augs, c, h, w = inputs.size()
        
        # 1. 壓平變成 (Batch * 5, 3, H, W) 餵給模型
        inputs = inputs.view(-1, c, h, w).to(device)
        
        # 2. 模型預測
        outputs = model(inputs) # (Batch * 5, num_classes)
        
        # 3. 轉成機率並取平均
        probs = F.softmax(outputs, dim=1)      # 先轉機率
        probs = probs.view(bs, n_augs, -1)     # 變回 (Batch, 5, num_classes)
        avg_probs = torch.mean(probs, dim=1)   # 對 5 次結果取平均 -> (Batch, num_classes)
        
        # 4. 取出最高分
        _, preds = torch.max(avg_probs, 1)
        
        # 5. 算對幾題
        tta_corrects += torch.sum(preds == labels.data)
        total_samples += labels.size(0)
        
        if (i+1) % 1== 0:
            print(f"進度: {i+1}/{len(tta_val_loader)} Batch...")

# ==========================================
# 5. 印出結果比較
# ==========================================
tta_acc = tta_corrects.double() / total_samples

print("-" * 50)
print(f"📊 驗證集總樣本數: {total_samples}")
print(f"✅ TTA 後的準確率 (Accuracy): {tta_acc:.4f}")
print("-" * 50)

# 如果你有存原本的 best_acc，可以比較一下
if 'best_acc' in globals():
    print(f"原本的最佳準確率 (Best Acc): {best_acc:.4f}")
    diff = tta_acc - best_acc
    if diff > 0:
        print(f"🚀 TTA 帶來了 +{diff:.4f} 的提升！")
    else:
        print(f"⚠️ TTA 沒有提升 ({diff:.4f})，可能變換太劇烈或模型已經很穩定了。")

In [ ]:
import pandas as pd
import os
import torch.nn.functional as F

print("=" * 50)
print("🚀 啟動 TTA (Test Time Augmentation) 超級預測模式...")
print("=" * 50)

# ==========================================
# 1. 定義 TTA 的變換策略
# ==========================================
# 我們定義 5 種看圖的方式
# 注意：這裡只做幾何變換，不做顏色變換 (ColorJitter)，以免誤導模型
inference_size = img_size # B1 的尺寸

def get_tta_transforms(size):
    return [
        # 1. 原圖 (標準)
        transforms.Compose([
            transforms.Resize((size, size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # 2. 水平翻轉
        transforms.Compose([
            transforms.Resize((size, size)),
            transforms.RandomHorizontalFlip(p=1.0), # 強制翻轉
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # 3. 稍微放大 (Center Crop)
        transforms.Compose([
            transforms.Resize(int(size * 1.1)), # 先放大一點
            transforms.CenterCrop(size),        # 再切中間
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # 4. 稍微向左轉 (Rotation)
        transforms.Compose([
            transforms.Resize((size, size)),
            transforms.RandomRotation(degrees=(10, 10)), # 固定轉 10 度
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # 5. 稍微向右轉
        transforms.Compose([
            transforms.Resize((size, size)),
            transforms.RandomRotation(degrees=(-10, -10)), # 固定轉 -10 度
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
    ]

tta_transforms = get_tta_transforms(inference_size)

# ==========================================
# 2. 修改 Dataset 以支援 TTA
# ==========================================
class TTATestDataset(Dataset):
    def __init__(self, test_dir, transforms_list):
        self.test_dir = test_dir
        self.transforms_list = transforms_list
        self.image_files = [f for f in os.listdir(test_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.test_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        # 回傳一個 list，包含這張圖的 5 種變身版本
        augmented_images = [t(image) for t in self.transforms_list]
        
        # 這裡我們要把 5 張圖疊在一起變成一個 Tensor: (5, 3, 240, 240)
        return torch.stack(augmented_images), img_name

# ==========================================
# 3. 執行 TTA 預測
# ==========================================
tta_dataset = TTATestDataset(TEST_DIR, tta_transforms)
# batch_size 要改小，因為現在每一個 item 其實包含了 5 張圖
tta_loader = DataLoader(tta_dataset, batch_size=4, shuffle=False, num_workers=2)

model.eval()
predictions = []
image_ids = []

print(f"開始預測... (每張圖會預測 {len(tta_transforms)} 次並取平均)")

with torch.no_grad():
    for i, (inputs, filenames) in enumerate(tta_loader):
        # inputs 的形狀是 (Batch, 5, 3, 240, 240)
        # 我們要把它壓平成 (Batch * 5, 3, 240, 240) 才能餵給模型
        bs, n_augs, c, h, w = inputs.size()
        inputs = inputs.view(-1, c, h, w).to(device)
        
        # 模型預測
        outputs = model(inputs) # (Batch * 5, 120)
        
        # 把這 5 次預測的機率平均起來
        # 1. 先把 Logits 轉成機率 (Softmax)
        probs = F.softmax(outputs, dim=1)
        # 2. 變回 (Batch, 5, 120)
        probs = probs.view(bs, n_augs, -1)
        # 3. 對 "5" 那個維度取平均 -> (Batch, 120)
        avg_probs = torch.mean(probs, dim=1)
        
        # 取出最高分的類別
        _, preds = torch.max(avg_probs, 1)
        
        predictions.extend(preds.cpu().numpy())
        image_ids.extend(filenames)
        
        if (i+1) % 10 == 0:
            print(f"Batch {i+1}/{len(tta_loader)} 完成")

# ==========================================
# 4. 存檔
# ==========================================
class_names = full_dataset.classes 
predicted_labels = [class_names[p] for p in predictions]

clean_ids = image_ids 

df_sub = pd.DataFrame({
    'id': clean_ids,
    'label': predicted_labels
})

submission_file = 'submission_tta.csv'
df_sub.to_csv(submission_file, index=False)

print(f"✅ TTA 預測完成！檔案已儲存為： {submission_file}")
print("下一步：請下載此檔案並上傳到 Kaggle，就能看到準確度了！")
print("-" * 50)
print("前 5 筆預測預覽：")
print(df_sub.head())

## 看圖片和模型預測的類別

In [ ]:
def visualize_model_predictions(model, dataloader, class_names, num_images=6):
    """
    隨機抓幾張圖出來，顯示預測結果 vs 真實答案
    """
    was_training = model.training # 記錄原本的模式
    model.eval() # 切換到評估模式
    images_so_far = 0
    
    cols = 3  # 一排顯示 3 張圖 (這樣 90 張剛好 10 排)
    rows = math.ceil(num_images / cols) # 自動算出需要幾排
    
    # 設定畫布大小：寬度固定 24，高度根據排數自動變長 (每排給 3.5 單位高)
    fig = plt.figure(figsize=(20, rows * 3.5))

    print("正在繪製預測結果...")

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size(0)):
                images_so_far += 1
                ax = plt.subplot(rows, cols, images_so_far)
                ax.axis('off')
                
                # --- 判斷對錯，決定顏色 ---
                predicted_label = class_names[preds[j]]
                img_name = filenames[j] # 顯示檔名
                
                title = f"Pred: {predicted_label}\nID: {img_name}"
                ax.set_title(title, color='black', fontsize=16)

                # --- 圖片還原 (Un-normalize) ---
                # 因為我們之前把圖片標準化過 (變得很暗)，現在要還原回來才看得到原圖
                img = inputs.cpu().data[j].numpy().transpose((1, 2, 0))
                mean = np.array([0.485, 0.456, 0.406])
                std = np.array([0.229, 0.224, 0.225])
                img = std * img + mean
                img = np.clip(img, 0, 1) # 確保數值在 0~1 之間
                
                plt.imshow(img)

                if images_so_far == num_images:
                    model.train(mode=was_training) # 恢復原本模式
                    plt.tight_layout()
                    plt.show()
                    return

    model.train(mode=was_training)

# --- 執行視覺化 ---

# 1. 取得類別名稱 (從 dataset 裡面抓)
# full_dataset.classes 會是一個 list，例如 ['Chihuahua', 'Japanese_spaniel', ...]
class_names = full_dataset.classes 

# 2. 呼叫函式
# num_images=6 代表顯示 6 張圖
visualize_model_predictions(model, test_loader, class_names, num_images=4)

## 注意力熱力圖

In [ ]:
def visualize_attention(model, dataloader, num_images=4):
    """
    使用 Grad-CAM 繪製模型的注意力熱力圖
    """
    model.eval()
    
    # 1. 指定我們要看哪一層的特徵圖
    # 對於 EfficientNet，通常看最後一個卷積層 (features 的最後一層)
    # 因為我們有改寫模型結構，所以是 model.features[-1]
    target_layers = [model.features[-1]]

    # 建立 GradCAM 物件
    cam = GradCAM(model=model, target_layers=target_layers)

    # 準備畫布
    fig = plt.figure(figsize=(16, num_images * 4))
    
    print("正在生成注意力熱力圖 (Grad-CAM)...")

    # 從 dataloader 抓一批圖出來
    data_iter = iter(dataloader)
    images, labels = next(data_iter) # 這裡是 val_loader，所以有 labels
    
    # 如果是 test_loader，改成 images, filenames = next(data_iter)
    # 然後把下面的 labels[i] 改掉即可

    for i in range(num_images):
        input_tensor = images[i].unsqueeze(0).to(device)
        
        # --- 2. 產生熱力圖 ---
        # targets=None 表示自動找分數最高的那個類別來看
        grayscale_cam = cam(input_tensor=input_tensor, targets=None)
        grayscale_cam = grayscale_cam[0, :] # 取出第一張圖的 heatmap

        # --- 3. 處理原圖 (還原成人類看得懂的樣子) ---
        img = images[i].cpu().numpy().transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        # --- 4. 疊加圖片 ---
        # show_cam_on_image 需要 float32 格式的圖片
        visualization = show_cam_on_image(img, grayscale_cam, use_rgb=True)

        # --- 5. 繪圖 ---
        # 左邊放原圖
        ax1 = plt.subplot(num_images, 2, 2*i + 1)
        ax1.imshow(img)
        ax1.set_title("Original Image")
        ax1.axis('off')

        # 右邊放熱力圖
        ax2 = plt.subplot(num_images, 2, 2*i + 2)
        ax2.imshow(visualization)
        ax2.set_title("Grad-CAM (Attention Map)")
        ax2.axis('off')
        
    plt.tight_layout()
    plt.show()

# --- 執行 ---
# 使用驗證集來看 (因為我們想知道它對這些狗的看法)
visualize_attention(model, test_loader, num_images=12)

## 模型 : EfficientNet b0
注意! : 
- acc 是透過 val_loader 來的，test_loader 需要上傳後才可以得到 acc
- 最終簡報應該要使用上傳後得到的結果才是準確的

未解凍 acc: 0.7 (大概，忘記有沒有跑了) <br>
解凍 acc: 0.75 (N=4) <br>
加上 schedular: 0.8 (印象中) <br>
加上 attention (CBAM): 0.8247 (可能是剛好那次驗證集很簡單) <br>
加上 weight_decay + label_smoothing: 0.8451 (印象中) <br>
加上 AutoAugment: 0.81 (可能是訓練集變太難，細微特徵學不太到) <br>

注意力熱力圖: <br>
目前還是可以發現注意力集中在狗身上，大部分在頭上 <br>
但如果有多隻狗勾的話，會變成一大片區域都注意 <br>
 <br>
接下來可以做 : 
1. TTA (混和圖片讓模型輸出可能機率，例如: 哈士奇、吉娃娃: 0.5、0.5)
2. 模型換成 efficientnet_b1、b3、b4 或 SOTA 或 ViT
3. 報告可以講說遇到甚麼問題，使用方法解決或沒解決，跟想像中不一樣的地方
4. 還有比較！！！

## 模型 : EfficientNet b1
注意! : 
- acc 是透過 val_loader 來的，test_loader 需要上傳後才可以得到 acc
- 最終簡報應該要使用上傳後得到的結果才是準確的


未解凍 acc: 0.79 <br>
解凍 acc:  0.82 <br>
加上 schedular: 0.84 <br>
加上 attention (CBAM): <br>
- 未解凍 : 0.8571
- 解凍(N=1) : 0.8496
- 全解凍 : 0.84

加上 weight_decay + label_smoothing: 0.8647 <br>
加上 AutoAugment: 0.83 (可能是訓練集變太難，細微特徵學不太到) <br>
TTA : 0.81 (沒用) <br>

注意力熱力圖: <br>
目前還是可以發現注意力集中在狗身上，大部分在頭上 <br>
但如果有多隻狗勾的話，會變成一大片區域都注意 <br>
有一張不知道為啥注意力跑到其中一片草地上 <br>

## 模型 : EfficientNet b3
注意! : 
- acc 是透過 val_loader 來的，test_loader 需要上傳後才可以得到 acc
- 最終簡報應該要使用上傳後得到的結果才是準確的

未解凍 acc: 0.8346 <br>
解凍 acc: 0.8496 (N=4)  <br>
加上 schedular: 0.8496 <br>
加上 attention (CBAM): <br>
- 未解凍 : 
- 解凍(N=4) : 0.87
- 全解凍 : 

加上 weight_decay + label_smoothing: 0.85 <br>
加上 AutoAugment: 0.85 (可能是訓練集變太難，細微特徵學不太到) <br>
TTA : 0.81 (沒用) <br>

注意力熱力圖: <br>
目前還是可以發現注意力集中在狗身上，大部分在頭上 <br>
但如果有多隻狗勾的話，會變成一大片區域都注意 <br>
有一張不知道為啥注意力跑到其中一片草地上 <br>

## 模型 : EfficientNet b4
注意! : 
- acc 是透過 val_loader 來的，test_loader 需要上傳後才可以得到 acc
- 最終簡報應該要使用上傳後得到的結果才是準確的

未解凍 acc: 0.8346 <br>
解凍 acc: 0.8496 (N=2)  <br>
加上 schedular: 0.8496 <br>
加上 attention (CBAM): <br>
- 未解凍 : 0.8571
- 解凍(N=2) : 0.8571
- 全解凍 : 

加上 weight_decay : 0.8647 <br>
加上 label_smoothing: 0.8721 <br>
加上 AutoAugment: 0.8496 (可能是訓練集變太難，細微特徵學不太到) <br>
TTA : 0.81 (沒用) <br>

注意力熱力圖: <br>
目前還是可以發現注意力集中在狗身上，大部分在頭上 <br>
但如果有多隻狗勾的話，會變成一大片區域都注意 <br>
有一張不知道為啥注意力跑到其中一片草地上 <br>

## 模型 : ResNet 50 
注意! : 
- acc 是透過 val_loader 來的，test_loader 需要上傳後才可以得到 acc
- 最終簡報應該要使用上傳後得到的結果才是準確的

未解凍 acc: 0.8647 <br>
解凍 acc: 
- 0.8195 (N=1)
- 0.7947 (N=2)
加上 schedular:
- 0.8647 (凍結)
- 0.8195 (N=1)
加上 attention (CBAM):
- 未解凍 : 0.8496
- 解凍(N=2) : 0.8872
- 全解凍 : 

加上 weight_decay :  <br>
加上 label_smoothing: 0.8722 <br>
加上 AutoAugment: 0.8571 (可能是訓練集變太難，細微特徵學不太到) <br>
TTA : 
- 0.8496 (沒用)
- 0.8496

注意力熱力圖: <br>
目前還是可以發現注意力集中在狗身上，大部分在頭上 <br>
但如果有多隻狗勾的話，會變成一大片區域都注意 <br>
有一張不知道為啥注意力跑到其中一片草地上 <br>